# Data Importing And Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHSe44usciuN2aOVJt0XzvNgDFwLbW2WefttlHS7x9Ht-6XAFp3W9Q
Mounted at /content/gdrive


In [2]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Datasets/Incidents_service.xlsx')
df.shape

(141712, 25)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 1.Feature Selection

In [4]:
df_new = df[[ 'count_updated', 'opened_by', 'Support_group', 'category_ID', 'ID_status','impact']]
df_new.shape

(141712, 6)

In [5]:
df_new = df_new[df_new.ID_status != -100]
df_new.ID_status.value_counts()

Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
Name: ID_status, dtype: int64

In [6]:
df_new = pd.get_dummies(df_new, columns=['Support_group','category_ID','ID_status','opened_by'], drop_first=True)
print(df_new.shape)
df_new.head()

(141707, 349)


,count_updated,impact,Support_group_Group 11,Support_group_Group 12,Support_group_Group 13,Support_group_Group 14,Support_group_Group 15,Support_group_Group 16,Support_group_Group 17,Support_group_Group 18,Support_group_Group 19,Support_group_Group 2,Support_group_Group 20,Support_group_Group 21,Support_group_Group 22,Support_group_Group 23,Support_group_Group 24,Support_group_Group 25,Support_group_Group 26,Support_group_Group 27,Support_group_Group 28,Support_group_Group 29,Support_group_Group 3,Support_group_Group 30,Support_group_Group 31,Support_group_Group 32,Support_group_Group 33,Support_group_Group 34,Support_group_Group 35,Support_group_Group 36,Support_group_Group 37,Support_group_Group 38,Support_group_Group 39,Support_group_Group 4,Support_group_Group 41,Support_group_Group 43,Support_group_Group 44,Support_group_Group 45,Support_group_Group 46,Support_group_Group 47,...,opened_by_Opened by 503,opened_by_Opened by 505,opened_by_Opened by 508,opened_by_Opened by 512,opened_by_Opened by 515,opened_by_Opened by 517,opened_by_Opened by 519,opened_by_Opened by 52,opened_by_Opened by 520,opened_by_Opened by 524,opened_by_Opened by 528,opened_by_Opened by 530,opened_by_Opened by 533,opened_by_Opened by 535,opened_by_Opened by 536,opened_by_Opened by 538,opened_by_Opened by 54,opened_by_Opened by 55,opened_by_Opened by 58,opened_by_Opened by 59,opened_by_Opened by 60,opened_by_Opened by 62,opened_by_Opened by 66,opened_by_Opened by 67,opened_by_Opened by 68,opened_by_Opened by 7,opened_by_Opened by 70,opened_by_Opened by 72,opened_by_Opened by 73,opened_by_Opened by 74,opened_by_Opened by 75,opened_by_Opened by 77,opened_by_Opened by 79,opened_by_Opened by 8,opened_by_Opened by 80,opened_by_Opened by 88,opened_by_Opened by 9,opened_by_Opened by 90,opened_by_Opened by 91,opened_by_Opened by 94
0,0,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,4,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2. Scaling

In [7]:
from sklearn.preprocessing import MinMaxScaler
mx_scaler = MinMaxScaler()
final_df2 = mx_scaler.fit_transform(df_new.drop(columns = 'impact'))
final_df2 = pd.DataFrame(final_df2, columns=df_new.drop(columns = 'impact').columns)
print(final_df2.head())

   count_updated  ...  opened_by_Opened by  94
0       0.000000  ...                      0.0
1       0.023256  ...                      0.0
2       0.031008  ...                      0.0
3       0.007752  ...                      0.0
4       0.015504  ...                      0.0

[5 rows x 348 columns]


# Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X = final_df2
y = df_new['impact']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101, stratify= y)
print(y_train.value_counts())
print(y_test.value_counts())

2 - Medium    94030
3 - Low        2720
1 - High       2444
Name: impact, dtype: int64
2 - Medium    40300
3 - Low        1166
1 - High       1047
Name: impact, dtype: int64


# Dimensionality Reduction LDA

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=10)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Algorithms

## 1. Decision Tree:

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dt = DecisionTreeClassifier(random_state=101)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9367487592030673
[[  319   708    20]
 [  760 39076   464]
 [   17   720   429]]
              precision    recall  f1-score   support

    1 - High       0.29      0.30      0.30      1047
  2 - Medium       0.96      0.97      0.97     40300
     3 - Low       0.47      0.37      0.41      1166

    accuracy                           0.94     42513
   macro avg       0.58      0.55      0.56     42513
weighted avg       0.93      0.94      0.94     42513



## 2. Random Forest:

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 500, random_state= 101)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9495213228894691
[[  243   793    11]
 [  275 39741   284]
 [   11   772   383]]
              precision    recall  f1-score   support

    1 - High       0.46      0.23      0.31      1047
  2 - Medium       0.96      0.99      0.97     40300
     3 - Low       0.56      0.33      0.42      1166

    accuracy                           0.95     42513
   macro avg       0.66      0.52      0.57     42513
weighted avg       0.94      0.95      0.94     42513



## 3.Support Vector Classifier

In [12]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
y_pred = svc_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9521558111636441
[[   99   944     4]
 [   47 40225    28]
 [    7  1004   155]]
              precision    recall  f1-score   support

    1 - High       0.65      0.09      0.17      1047
  2 - Medium       0.95      1.00      0.98     40300
     3 - Low       0.83      0.13      0.23      1166

    accuracy                           0.95     42513
   macro avg       0.81      0.41      0.46     42513
weighted avg       0.94      0.95      0.94     42513



## 4. Gausian NB

In [13]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9167078305459506
[[  313   697    37]
 [  679 37778  1843]
 [   15   270   881]]
              precision    recall  f1-score   support

    1 - High       0.31      0.30      0.30      1047
  2 - Medium       0.98      0.94      0.96     40300
     3 - Low       0.32      0.76      0.45      1166

    accuracy                           0.92     42513
   macro avg       0.53      0.66      0.57     42513
weighted avg       0.94      0.92      0.93     42513



## 5. KNN

In [14]:
from sklearn.neighbors  import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9471926234328323
[[  301   740     6]
 [  367 39525   408]
 [   16   708   442]]
              precision    recall  f1-score   support

    1 - High       0.44      0.29      0.35      1047
  2 - Medium       0.96      0.98      0.97     40300
     3 - Low       0.52      0.38      0.44      1166

    accuracy                           0.95     42513
   macro avg       0.64      0.55      0.59     42513
weighted avg       0.94      0.95      0.94     42513



#Meta Algorithms

## 1. Voting Classifier

In [15]:
from sklearn.ensemble import VotingClassifier
nb = GaussianNB()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(random_state=101)
# Define a list called classifier that contains the tuples (classifier_name, classifier)
classifiers = [('GaussianNB', nb),
('K Nearest Neighbours', knn),
('Classification Tree', dt)]
# Iterate over the defined list of tuples containing the classifiers
for clf_name, clf in classifiers:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

vc = VotingClassifier(estimators=classifiers)
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)
print('Voting Classifier: {:.3f}'.format(accuracy_score(y_test, y_pred)))

GaussianNB : 0.917
K Nearest Neighbours : 0.950
Classification Tree : 0.937
Voting Classifier: 0.947


## 2. Bagging Classifier

In [16]:
from sklearn.ensemble import BaggingClassifier
dt = DecisionTreeClassifier(random_state=101)
bc = BaggingClassifier(base_estimator=dt, n_estimators=300, n_jobs=-1)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.949


## 3. AdaBoost Classifier

In [17]:
from sklearn.ensemble import AdaBoostClassifier
dt = DecisionTreeClassifier(random_state=101)
abc = AdaBoostClassifier(base_estimator=dt, n_estimators=300)
abc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.949


## 4. XGBoost Classifier

In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.953
